Author: Krish

In [2]:
# Importing relevant libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

### User input required
Put the data path on your system in the cell below

Note: Assumes data has been restricted only to datasets that included the 'Termination Reason' column 

In [3]:
data_path = "/Users/viviadams/Downloads/CAR_Includes_Termination"

### User input ends

### Reading all filenames in the data folder

In [4]:
folder = Path(data_path)
files = sorted(list(folder.glob("*.csv")) + list(folder.glob("*.xlsx")))
df_main = pd.DataFrame(columns=['Contact Session ID', 'EP Name', 'Flow Name', 'Activity Name', 'Activity Start Timestamp', 
                                'Queue Name', 'Agent Name', 'Termination Reason'])
df_main

,Contact Session ID,EP Name,Flow Name,Activity Name,Activity Start Timestamp,Queue Name,Agent Name,Termination Reason


### Reading all data files
The code chunk below reads and appends all the CAR data files. The first two rows of each file are blank and thus ignored.

In [5]:
i=0
for f in files:
    i = i + 1
    if f.suffix.lower() == ".csv":
        df = pd.read_csv(f, header=2, dtype=str, engine="python", skip_blank_lines=False)
    else:  # .xlsx
        df = pd.read_excel(f, sheet_name=0, header=2, dtype=str)
    df_main = pd.concat([df_main, df], ignore_index=True)
    print(i, f.stem, df.shape)

1 CAR - EP, Flow, Activity, Queue, &amp; Agent Names 3-16 to 3-22-25 (45326, 8)
2 CAR - EP, Flow, Activity, Queue, &amp; Agent Names 3-30 to 4-5-25 (45072, 8)
3 CAR - EP, Flow, Activity, Queue, &amp; Agent Names 4-13 to 4-19-25 (41088, 8)
4 CAR - EP, Flow, Activity, Queue, &amp; Agent Names 4-20 to 4-26-25 (46686, 8)
5 CAR - EP, Flow, Activity, Queue, &amp; Agent Names 4-27 to 5-3-25 (44169, 8)
6 CAR - EP, Flow, Activity, Queue, &amp; Agent Names 4-6 to 4-12-25 (46630, 8)
7 CAR - EP, Flow, Activity, Queue, &amp; Agent Names 5-11 to 5-17-25 (43268, 8)
8 CAR - EP, Flow, Activity, Queue, &amp; Agent Names 5-18 to 5-24-25 (45104, 8)
9 CAR - EP, Flow, Activity, Queue, &amp; Agent Names 5-25 to 5-31-25 (36851, 8)
10 CAR - EP, Flow, Activity, Queue, &amp; Agent Names 5-4 to 5-10-25 (43955, 8)
11 CAR - EP, Flow, Activity, Queue, &amp; Agent Names 6-1 to 6-7-25 (44523, 8)
12 CAR - EP, Flow, Activity, Queue, &amp; Agent Names 6-15 to 6-21-25 (37234, 8)
13 CAR - EP, Flow, Activity, Queue, &amp; A

### Time datatype conversion
The code chunk below converts time from string to datetime datatype.

In [6]:
df_main["Activity Start Timestamp"] = df_main["Activity Start Timestamp"].apply(
    lambda x: datetime.strptime(x, "%Y/%m/%d %I:%M:%S %p"))

In [7]:
# Checking the datatype of all columns
df_main.dtypes

Contact Session ID                  object
EP Name                             object
Flow Name                           object
Activity Name                       object
Activity Start Timestamp    datetime64[ns]
Queue Name                          object
Agent Name                          object
Termination Reason                  object
dtype: object

In [8]:
# Creating a new column 'hour' as it will be useful to visualize peak calling hours
df_main["hour"] = df_main["Activity Start Timestamp"].dt.hour


### Possible Flow Name Values

In [30]:
df_main['Flow Name'].unique()

array([nan, 'LACMain', 'ClosedHoursHolidaysMenu', 'FarmworkerMain',
       'PreLegalMenuSeniorsMenu', 'LegalMenu', 'LegalFamilyMenu',
       'Queues', 'CourtesyCallback', 'LegalHousingMenu',
       'LegalEmploymentMenu', 'LegalBenefitsMenu', 'OtherLegalMenu',
       'LegalImmigrationMenu', 'Intake_Outdial', 'ClosedQueueMenu',
       'LegalHIVMenu'], dtype=object)

### Possible Queue Name Values  

In [17]:
df_main['Queue Name'].unique()

array([nan, 'Consumer SP', 'Staff Directory English Transfer',
       'Family SP', 'Staff Directory Spanish Transfer', 'Family',
       'Consumer', 'Employment', 'Benefits', 'Clinic Voicemail Transfer',
       'Immigration SP', 'Intake Outdial Queue', 'Front Desk Transfer',
       'SubSenior Benefits', 'SubSenior Tenant', 'SubSenior Homeowner',
       'HIV Voicemail Transfer', 'SubSenior Other',
       'Trafficking Voicemail Transfer', 'SubSenior Family',
       'Veterans Benefits Voicemail Transfer',
       'Criminal Records Voicemail Transfer', 'Housing',
       'SubSenior Consumer', 'Housing SubSeniors', 'ADAPT', 'Benefits SP',
       'SubSenior ADAPT', 'ADAPT SubSeniors', 'Other SubSeniors',
       'Education SP', 'SubSenior Employment', 'SubSenior Benefits SP',
       'Education', 'Farmworker Voicemail Transfer',
       'SubSenior Consumer SP', 'ADAPT SP', 'Immigration',
       'SubSenior Other SP', 'SubSenior Family SP', 'Employment SP',
       'Homeowner SubSeniors', 'Consumer S

### Possible EP Name Values 

In [18]:
df_main['EP Name'].unique()

array(['Main Number Telephony EP', nan, 'Courtesy Callback Telephony EP',
       'Closed Hours-Holidays Menu Telephony EP',
       'Farmworker Main Number Telephony EP',
       'Pre-Legal Menu Seniors Menu Telephony EP',
       'Legal Menu Telephony EP', 'Legal Family Menu Telephony EP',
       'All LAC Queues Telephony EP', 'Legal Housing Menu Telephony EP',
       'Legal Employment Menu Telephony EP',
       'Legal Benefits Menu Telephony EP',
       'Other Legal Menu Telephony EP',
       'Legal Immigration Menu Telephony EP', 'Intake Outdial EP',
       'Closed Queue Menu Telephony EP', 'Legal HIV Menu Telephony EP'],
      dtype=object)

### Filling in Missing Values in Columns of Interest 

In [53]:
df_main['Queue Name'] = df_main['Queue Name'].fillna('Unknown Queue')

df_main['EP Name'] = df_main['EP Name'].fillna('Unknown EP')

df_main['Termination Reason'] = df_main['Termination Reason'].fillna('Unknown Termination Reason')

### Total Unique Calls for Each Queue & EP Combination 

In [60]:
# Grouping by 'EP Name' and 'Queue Name' - only including unique Contact Session IDs 
calls_by_queue = pd.DataFrame(df_main.groupby(['EP Name','Queue Name'])['Contact Session ID'].nunique().reset_index(name='Total Calls'))

calls_by_queue.head(15)

,EP Name,Queue Name,Total Calls
0,All LAC Queues Telephony EP,ADAPT,154
1,All LAC Queues Telephony EP,ADAPT SP,16
2,All LAC Queues Telephony EP,ADAPT SubSeniors,7
3,All LAC Queues Telephony EP,ADAPT SubSeniors SP,1
4,All LAC Queues Telephony EP,Benefits,314
5,All LAC Queues Telephony EP,Benefits SP,32
6,All LAC Queues Telephony EP,Benefits SubSeniors,7
7,All LAC Queues Telephony EP,Benefits SubSeniors SP,4
8,All LAC Queues Telephony EP,Consumer,701
9,All LAC Queues Telephony EP,Consumer SP,50


### Abandoned Calls for Each EP & Queue 

In [47]:
 # Removing duplicate entries so only include the final entry for each Contact Session ID 
last_instance = pd.DataFrame(df_main.drop_duplicates(subset=['Contact Session ID'], keep='last'))

    # Filtering to only include calls where it ended because the customer left 
abandoned_calls = last_instance[last_instance['Termination Reason'] == 'Customer Left']

    # Grouping by 'EP Name' and 'Queue Name' 
aban_calls_df = pd.DataFrame(abandoned_calls.groupby(['EP Name', 'Queue Name'])['Contact Session ID'].nunique().reset_index(name='Num Calls')) 

aban_calls_df.head(15)

,EP Name,Queue Name,Num Calls
0,All LAC Queues Telephony EP,ADAPT,20
1,All LAC Queues Telephony EP,ADAPT SP,5
2,All LAC Queues Telephony EP,ADAPT SubSeniors,2
3,All LAC Queues Telephony EP,Benefits,36
4,All LAC Queues Telephony EP,Benefits SP,2
5,All LAC Queues Telephony EP,Benefits SubSeniors,2
6,All LAC Queues Telephony EP,Consumer,72
7,All LAC Queues Telephony EP,Consumer SP,5
8,All LAC Queues Telephony EP,Consumer SubSeniors,2
9,All LAC Queues Telephony EP,Education,3


### Total Open Queue Calls for Each Queue 

In [52]:
# Filtering the dataframe to only include rows with a Queue Name and without a termination reason 
open_queue_calls = df_main[~df_main['Queue Name'].isna() & df_main['Termination Reason'].isna()]


# The unique number of Contact Session IDs is less than the number of entries with a Queue Name 
print(open_queue_calls['Contact Session ID'].nunique())
print(open_queue_calls.shape)

 # Removing duplicate entries so only include the final entry for each Contact Session ID 
uniq_open_queue_call = pd.DataFrame(open_queue_calls.drop_duplicates(subset=['Contact Session ID'], keep='last'))

# Checking that the number of Contact Session IDs and number of entries match 
print(uniq_open_queue_call['Contact Session ID'].nunique())
print(uniq_open_queue_call.shape)


grouped_uniq_open_queue = pd.DataFrame(uniq_open_queue_call.groupby(['EP Name','Queue Name'])['Contact Session ID'].nunique().reset_index(name='Num Calls'))

grouped_uniq_open_queue.head(15)


33597
(110354, 9)
33597
(33597, 9)


,EP Name,Queue Name,Num Calls
0,All LAC Queues Telephony EP,ADAPT,22
1,All LAC Queues Telephony EP,ADAPT SP,5
2,All LAC Queues Telephony EP,ADAPT SubSeniors,2
3,All LAC Queues Telephony EP,Benefits,37
4,All LAC Queues Telephony EP,Benefits SP,2
5,All LAC Queues Telephony EP,Benefits SubSeniors,2
6,All LAC Queues Telephony EP,Consumer,74
7,All LAC Queues Telephony EP,Consumer SP,5
8,All LAC Queues Telephony EP,Consumer SubSeniors,2
9,All LAC Queues Telephony EP,Education,3
